#Load Dataset

In [1]:
!wget -O LLM-ReDial-2024.zip "https://www.dropbox.com/scl/fi/x9avfdx2a1k6uq97f0efj/LLM-ReDial-2024.zip?rlkey=ijqpf91d13d6lowek3ebjvd0n&e=2&dl=1"
!unzip LLM-ReDial-2024.zip
!unzip LLM_Redial.zip
!rm LLM_Redial.zip
!rm LLM-ReDial-2024.zip
!rm -rf __MACOSX

--2024-12-04 14:05:56--  https://www.dropbox.com/scl/fi/x9avfdx2a1k6uq97f0efj/LLM-ReDial-2024.zip?rlkey=ijqpf91d13d6lowek3ebjvd0n&e=2&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4ff8e7e0cf21d94cd02e9e4ee0.dl.dropboxusercontent.com/cd/0/inline/Cfmnsvi33V2mjdM7Kaa8tkajMhDHSegwzxj9DqmIOKZ9TKWwO8LNGNIUoXVrN0TkgdMpB4siTCU_wSyIbJyLLX6lqhxM9hYnXI9AeUDxKGn0HHAUhyZAsum63S_e7iESB2s/file?dl=1# [following]
--2024-12-04 14:05:57--  https://uc4ff8e7e0cf21d94cd02e9e4ee0.dl.dropboxusercontent.com/cd/0/inline/Cfmnsvi33V2mjdM7Kaa8tkajMhDHSegwzxj9DqmIOKZ9TKWwO8LNGNIUoXVrN0TkgdMpB4siTCU_wSyIbJyLLX6lqhxM9hYnXI9AeUDxKGn0HHAUhyZAsum63S_e7iESB2s/file?dl=1
Resolving uc4ff8e7e0cf21d94cd02e9e4ee0.dl.dropboxusercontent.com (uc4ff8e7e0cf21d94cd02e9e4ee0.dl.dropboxusercontent.com)... 162.125.81.15, 2620:100:6035:15::

#Load HF model

In [2]:
from huggingface_hub import notebook_login

notebook_login()



In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-b4d7dd08-c583-dc1e-9c0f-c7c2d80e1195)


In [3]:
!pip install transformers[torch]
!pip install bitsandbytes
!pip install peft
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 18.7 MB/s eta 0:00:00


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

In [5]:
model_id = "meta-llama/Llama-3.2-3B-Instruct"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [6]:
text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [7]:
def generate_zeroshot(conversation_id, conversations):
    conversation = "\n".join(conversations[conversation_id]["messages"])
    content = (
        "Pretend you are a movie recommender system. "
        "I will give you a conversation between a human and assistant. "
        "Based on the conversation, you reply me with 10 recommendations, "
        "including only their titles without extra sentences or description. "
        f"Here is the conversation: {conversation}."
    )
    chat = [
        {"role": "user", "content": content}
    ]
    return tokenizer.apply_chat_template(chat, tokenize=False)

In [8]:
from Tools import read_user_data
path_movie_data = "/content/Movie/final_data.jsonl"

In [9]:
import json
item_map_path = "/content/Movie/item_map.json"
item_map = json.load(open(item_map_path,'r'))
user_map_path = "/content/Movie/user_ids.json"
user_map = json.load(open(user_map_path,'r'))

In [10]:
conversations_path = "/content/Movie/Conversation.txt"

conversations = []
current_conversation = []
conversation_id = 0
with open(conversations_path, 'r') as file:
    for line in file:
        line = line.strip()
        if not line:
            continue

        if line.isdigit():
            if current_conversation:
                conversations.append({
                    'conversation_id': conversation_id,
                    'messages': current_conversation
                })
            conversation_id = int(line)
            current_conversation = []
        else:
            current_conversation.append(line)

    if current_conversation:
        conversations.append({
            'conversation_id': conversation_id,
            'messages': current_conversation
        })

In [11]:
import random
user_ids = list(user_map.keys())
sample_percent = 0.01 # 10%

sample_size = int(sample_percent * len(user_ids))
sampled_users = random.sample(user_ids,sample_size)

In [12]:
dataset = []
for user_id in user_ids:
  user_data = read_user_data(path_movie_data, user_id)
  for conv_number, conversation in enumerate(user_data['Conversation']):
    conversation_id = conversation[f"conversation_{conv_number+1}"]["conversation_id"]
    prompt = generate_zeroshot(conversation_id, conversations)
    dataset.append({
        "user_id": user_id,
        "conversation_id": conversation_id,
        "user_might_like": user_data['user_might_like'],
        "prompt": prompt
    })

In [13]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(dataset, test_size=0.2, random_state=42)
print("Train prompts:", len(train_set))
print("Test prompts:", len(test_set))

Train prompts: 8071
Test prompts: 2018


In [14]:
prompts = [item['prompt'] for item in test_set[:500]]
results = text_generator(prompts, max_new_tokens=200, return_full_text=False)


In [15]:
answers = [result[0]['generated_text'] for result in results]
for answer in answers[:10]:
  print(answer)


assistant

Here are 10 movie recommendations based on the conversation:

1. The Hangover
2. The Big Bang Theory
3. The Office (US)
4. Parks and Recreation
5. Brooklyn Nine-Nine
6. Superbad
7. Bridesmaids
8. Monty Python and the Holy Grail
9. The IT Crowd
10. Arrested Development
assistant

Here are 10 movie recommendations:

1. The Man Who Shot Liberty Valance
2. The Great Gatsby
3. The Long Goodbye
4. The Shootist
5. The Valley of the Dolls
6. The Last Picture Show
7. The Man Who Came to Dinner
8. The Wild Bunch
9. The Nickelodeon
10. The Set-Up
assistant

1. The Angry Birds Movie
2. Star Wars: The Force Awakens
3. The Wind Rises
4. The Great Mouse Detective
5. Kiki's Delivery Service
6. The Secret Life of Arrietty
7. The Tale of The Princess Kaguya
8. Ponyo
9. Howl's Moving Castle
10. Spirited Away
assistant

1. Cleopatra
2. The Honeymooners Classic 39 Collection, Vol. 1: TV or Not TV, Honeymooners Highlights VHS
3. Annapolis
4. Full Metal Jacket
5. Rocky
6. The Karate Kid
7. Raging 

In [16]:
print(len(answers))

500


In [17]:
import re

def parse_recommendations(answer):
  pattern = r'\d+\.\s+"*([^"]+?)"*(?:\s\(\d{4}\))?(?:\s*-\s*.+)?(?:\s*\(\d-\d\))?$'
  recommendations = re.findall(pattern, answer, re.MULTILINE)
  return recommendations

In [18]:
import numpy as np

def recall_at_k(r, k, m):
    return (np.asarray(r)[:k] != 0).sum() / m

def dcg_at_k(r, k):
    r = np.asfarray(r)[:k]
    if r.size:
        return np.sum(np.subtract(np.power(2, r), 1) / np.log2(np.arange(2, r.size + 2)))
    return 0.

def idcg_at_k(k):
    return dcg_at_k(np.ones(k), k)

def ndcg_at_k(r, k, max_relevant):
    idcg = idcg_at_k(min(k, max_relevant))
    if not idcg:
        return 0.
    return dcg_at_k(r, k) / idcg

In [19]:
from sentence_transformers import SentenceTransformer

def embedding_init():
  sbert = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
  return sbert

def similarity_score(sbert, movies):
  embed1 = sbert.encode(movies[0])
  embed2 = sbert.encode(movies[1])
  return sbert.similarity(embed1,embed2).numpy()[0][0]

In [20]:
def calculate_similarities(sbert, user_might_like, recommendations):
  might_like_embeddings = sbert.encode(user_might_like)
  recommendations_embeddings = sbert.encode(recommendations)
  return sbert.similarity(might_like_embeddings, recommendations_embeddings).numpy()

def calculate_rel_vector(similarities, threshold=0.75):
  return np.any(np.array(similarities) >= threshold, axis=0)

In [21]:
def evaluate_model(dataset, answers, sbert):
    assert len(dataset) == len(answers)

    mean_ndcg = 0
    mean_recall = 0
    recommendations_count = 0
    for i in range(len(answers)):
       recommendations = parse_recommendations(answers[i])
       if len(recommendations) != 10:
         continue
       user_might_like = [item_map[id] for id in dataset[i]["user_might_like"]]
       similarities = calculate_similarities(sbert, user_might_like, recommendations)
       rel_vector = calculate_rel_vector(similarities)


       mean_ndcg += ndcg_at_k(rel_vector, 10, len(user_might_like))
       mean_recall += recall_at_k(rel_vector, 10, len(user_might_like))
       recommendations_count += 1

    mean_ndcg /= recommendations_count
    mean_recall /= recommendations_count

    return mean_ndcg, mean_recall

In [22]:
sbert = embedding_init()
ndcg, recall = evaluate_model(test_set[:500], answers, sbert)
print(f"NDCG: {ndcg:.4f}")
print(f"Recall: {recall:.4f}")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

NDCG: 0.1469
Recall: 0.1669
